# 1. ETL, Extracción Transformación y Carga de Datos

En este proceso extraemos los datos necesarios y los limpiamos para tenerlos en formato correcto para nuestros fines.

In [21]:
import pandas as pd
import json
import ast
import warnings
from io import StringIO
import hashlib
import matplotlib.pyplot as plt

from typing import List, Dict
import base64, csv

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import plotly.express as px
import seaborn as sns


warnings.filterwarnings('ignore')

In [22]:
# Show all properties on display and set style
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
warnings.simplefilter("ignore")

In [5]:
def showPie(columna):
  count_values = pd.Series(columna).value_counts()
  if len(count_values) > 15:
    count_values = count_values.iloc[0:15]
  datos = pd.DataFrame({"valor":count_values.index, "ocurrencia": count_values.values})

  plt.title(columna.name)
  plt.pie(datos["ocurrencia"], labels=datos['valor'], autopct='%1.1f%%')
  plt.show()

def concatenar(data_1, data_2,  axis=1):
  return pd.concat([data_1, data_2], axis=axis)

def contar_nulos(data):
  return data.isna().sum()

def mapear(columna: pd.Series, mapa={'NO': 0, 'SI':1}):
  return columna.map(mapa)

def showPiePx(columna, max=15):
  count_values = pd.Series(columna).value_counts()
  if len(count_values) > max:
    count_values = count_values.iloc[0:max]
  datos = pd.DataFrame({"valor":count_values.index, "ocurrencia": count_values.values})
  fig = px.pie(datos, values='ocurrencia', names='valor', title=columna.name)
  fig.update_traces(textposition='outside', textinfo='percent+label')
  fig.show()

In [ ]:
# Show all properties on display and set style
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
warnings.simplefilter("ignore")

In [8]:
URL_STEAM_GAMES = 'datasets/origin/output_steam_games.json'
URL_USERS_ITEMS = 'datasets/australian_users_items.json'
URL_USERS_REVIEWS = 'datasets/australian_user_reviews.json'

## 1. ETL de Dataset Steam Games

In [42]:
df_games_all = pd.read_json(URL_STEAM_GAMES, lines=True)

In [43]:
df_games_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [44]:
df_games_all.shape

(120445, 13)

In [45]:
df_games_all.isna().sum(axis=1).value_counts().sort_values(ascending=False)

13    88310
0     22530
1      6070
5      1940
3       733
4       391
2       349
6       121
10        1
Name: count, dtype: int64

Revisando los valores nulos a lo largo del eje 1, vemos que hay un número muy grande (88310) de filas completamente vacías, ya que tiene 13 columnas el dataset y la cuenta de nulos es igual a ese valor.

In [46]:
df_games = df_games_all[df_games_all.isna().sum(axis=1) != 13]

In [56]:
df_games_all[df_games_all.isna().sum(axis=1) == 6]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88346,None,None,Kingdom Watcher,None,http://store.steampowered.com/app/680970/Kingd...,None,"[Action, Free to Play, Strategy]",http://steamcommunity.com/app/680970/reviews/?...,"[Single-player, Online Multi-Player, HTC Vive,...",None,0.0,680970.0,None
88355,None,None,Panoptic,None,http://store.steampowered.com/app/541930/Panop...,None,"[Early Access, Indie, Action, VR]",http://steamcommunity.com/app/541930/reviews/?...,"[Multi-player, Local Multi-Player, Partial Con...",None,1.0,541930.0,None
88363,None,None,Golf Masters,None,http://store.steampowered.com/app/374970/Golf_...,None,"[Simulation, Sports, Casual, Indie, Golf, VR]",http://steamcommunity.com/app/374970/reviews/?...,"[Single-player, Steam Achievements, Partial Co...",None,0.0,374970.0,None
90852,None,None,NoLimits 2 Roller Coaster Simulation Demo,None,http://store.steampowered.com/app/319260/NoLim...,None,"[Simulation, Indie]",http://steamcommunity.com/app/319260/reviews/?...,"[Single-player, Game demo, HTC Vive, Oculus Ri...",None,0.0,319260.0,None
93683,None,None,BrainBread 2 Mod Tools,BrainBread 2 Mod Tools,http://store.steampowered.com/app/382990/Brain...,2016-02-17,None,http://steamcommunity.com/app/382990/reviews/?...,None,None,0.0,382990.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120246,None,None,Fruit Golf,None,http://store.steampowered.com/app/406890/Fruit...,None,"[Casual, Sports, Indie, VR]",http://steamcommunity.com/app/406890/reviews/?...,"[Single-player, Multi-player, Shared/Split Scr...",None,0.0,406890.0,None
120302,None,None,Mutato Match,None,http://store.steampowered.com/app/391480/Mutat...,None,"[Early Access, Indie, Casual, VR]",http://steamcommunity.com/app/391480/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",None,1.0,391480.0,None
120364,None,None,VR Dunhuang,None,http://store.steampowered.com/app/752200/VR_Du...,None,"[Adventure, RPG, Casual, Simulation]",http://steamcommunity.com/app/752200/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",None,0.0,752200.0,None
120367,None,None,COMPLEX a VR Puzzle Game,None,http://store.steampowered.com/app/772420/COMPL...,None,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/772420/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",None,0.0,772420.0,None


In [58]:
df_games.title.isna().sum()

2050

In [48]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [49]:
df_games.describe(include='object').T

,count,unique,top,freq
publisher,24083,8239,Ubisoft,385
genres,28852,883,[Action],1880
app_name,32133,32094,Soundtrack,3
title,30085,30054,Soundtrack,3
url,32135,32135,http://store.steampowered.com/app/761140/Lost_...,1
release_date,30068,3582,2012-10-16,100
tags,31972,15395,"[Casual, Simulation]",1292
reviews_url,32133,32132,http://steamcommunity.com/app/612880/reviews/?...,2
specs,31465,4649,[Single-player],2794
price,30758.0,162.0,4.99,4278.0


In [50]:
df_games['hash'] = df_games.apply(lambda row: hashlib.md5(row.astype(str).values.tobytes()).hexdigest(), axis=1)

In [51]:
df_games[df_games['hash'].duplicated()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,hash


In [52]:
df_games = df_games.drop('hash', axis=1)

In [ ]:
df_games[df_games.title.isna()]

In [60]:
df_games[df_games.app_name.isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160.0,None


In [61]:
try:
    df_games = df_games.drop(88384, axis=0)
except:
    pass

In [62]:
df_games.loc[90890, 'app_name'] = 'Duet'
df_games.loc[90890]

publisher                                                    None
genres                                            [Action, Indie]
app_name                                                     Duet
title                                                        None
url                   http://store.steampowered.com/app/317160/_/
release_date                                           2014-08-26
tags                                              [Action, Indie]
reviews_url     http://steamcommunity.com/app/317160/reviews/?...
specs                                  [Single-player, Game demo]
price                                                        None
early_access                                                  0.0
id                                                       317160.0
developer                                                    None
Name: 90890, dtype: object

Resetear el Indice

In [63]:
df_games = df_games.reset_index()

In [65]:
df_games = df_games.drop('index', axis=1);

In [66]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32134 non-null  object 
 3   title         30085 non-null  object 
 4   url           32134 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30757 non-null  object 
 10  early_access  32134 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


3. Eliminamos columnas consideradas innecesarias para el análisis

In [68]:
df_games = df_games.drop(['url', 'reviews_url', 'title'], axis=1)

In [69]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134 entries, 0 to 32133
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32134 non-null  object 
 3   release_date  30068 non-null  object 
 4   tags          31972 non-null  object 
 5   specs         31465 non-null  object 
 6   price         30757 non-null  object 
 7   early_access  32134 non-null  float64
 8   id            32133 non-null  float64
 9   developer     28836 non-null  object 
dtypes: float64(2), object(8)
memory usage: 2.5+ MB


4. Buscamos solucionar valores nulos en el id, con el fin de utilizarlo como identificador del juego

In [71]:
df_games[df_games.id.isna()]

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer
30960,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...","[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [72]:
df_games.loc[30960,'id'] = 200260

In [73]:
df_games['id'] =df_games['id'].astype('int').values

In [74]:
df_games[df_games.id.isna()]

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer


In [75]:
cuenta_duplicados = df_games.id.value_counts()
id_dups = cuenta_duplicados[cuenta_duplicados.values > 1].keys()
to_b = []

for i in id_dups:
  to_b.append(df_games[df_games['id'] == i].id.idxmax())

df_games.drop(to_b, axis=0, inplace=True)

In [76]:
df_games.query("id == 612880.0")

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer
14572,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...","[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880,Machine Games


In [77]:
df_games['hash'] = df_games.apply(lambda row: hashlib.md5(row.astype(str).values.tobytes()).hexdigest(), axis=1)

In [79]:
df_games['hash'].duplicated().sum()

0

5. Trabajamos con las columnas

Early Access

In [80]:
df_games.early_access = df_games.early_access.astype('bool').values

In [81]:
def isnumber(x):
    try:
        x = float(x)
        return x
    except:
        return 0

Price

In [82]:
df_games.price = df_games.price.apply(isnumber)

In [85]:
df_games['genres'].explode().unique().to_csv('generos.csv', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [86]:
df_games['genres'].explode().unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', None, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [87]:
sin_genero = df_games.genres.isna()

In [88]:
df_games['genres_tag'] = df_games['genres'] + df_games['tags']

In [94]:
len(df_games['genres_tag'].explode().unique())

340

In [98]:
generos_tags = []

In [104]:
df_games['genres_tag']

0        [Action, Casual, Indie, Simulation, Strategy, ...
1        [Free to Play, Indie, RPG, Strategy, Free to P...
2        [Casual, Free to Play, Indie, Simulation, Spor...
3        [Action, Adventure, Casual, Action, Adventure,...
4                                                      NaN
                               ...                        
32129    [Casual, Indie, Simulation, Strategy, Strategy...
32130    [Casual, Indie, Strategy, Strategy, Indie, Cas...
32131    [Indie, Racing, Simulation, Indie, Simulation,...
32132    [Casual, Indie, Indie, Casual, Puzzle, Singlep...
32133                                                  NaN
Name: genres_tag, Length: 32132, dtype: object

In [119]:
df_games.query("id == 768800")

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer,hash,genres_tag
9,RewindApp,"[Casual, Indie, Racing, Simulation]",Race,2018-01-04,"[Indie, Casual, Simulation, Racing]","[Single-player, Multi-player, Partial Controll...",0.0,False,768800,RewindApp,46f1529a82a5774fcaef506fed645654,"[Casual, Indie, Racing, Simulation, Indie, Cas..."


In [134]:
generos_tags = []

for i in df_games.index:
    lista_gt = df_games.loc[i, 'genres_tag']
    try:
        generos_tags.append([df_games.loc[i, 'id'],*set(lista_gt)])
    except:
        pass
        
    # if lista_gt != None:
    #     try:
    #         for j, l in enumerate(lista_gt):
    #             generos_tags.append([df_games.loc[i, 'id'], l['genres_tag']])
    #     except:
    #         pass

# generos_tags

In [140]:
tuplas = []
for i in generos_tags:
    for ind, j in enumerate(i):
        if ind > 0:
            tupla = [i[0], j]
            tuplas.append(tupla)

tuplas

[[761140, 'Indie'],
 [761140, 'Strategy'],
 [761140, 'Simulation'],
 [761140, 'Casual'],
 [761140, 'Action'],
 [643980, 'Trading Card Game'],
 [643980, 'Design & Illustration'],
 [643980, 'Difficult'],
 [643980, 'Tactical'],
 [643980, '2D'],
 [643980, 'Dark Fantasy'],
 [643980, 'Character Customization'],
 [643980, 'Replay Value'],
 [643980, 'Board Game'],
 [643980, 'Female Protagonist'],
 [643980, 'Indie'],
 [643980, 'PvP'],
 [643980, 'Card Game'],
 [643980, 'RPG'],
 [643980, 'Fantasy'],
 [643980, 'Turn-Based'],
 [643980, 'Strategy'],
 [643980, 'Competitive'],
 [643980, 'Free to Play'],
 [670290, 'Indie'],
 [670290, 'Multiplayer'],
 [670290, 'Sports'],
 [670290, 'Simulation'],
 [670290, 'Casual'],
 [670290, 'Free to Play'],
 [767400, 'Casual'],
 [767400, 'Adventure'],
 [767400, 'Action'],
 [772540, 'Shooter'],
 [772540, 'FPS'],
 [772540, 'Adventure'],
 [772540, 'Third Person'],
 [772540, 'Simulation'],
 [772540, 'Third-Person Shooter'],
 [772540, 'Sniper'],
 [772540, 'Action'],
 [7742

In [145]:
df_juego_genero_tag = pd.DataFrame(tuplas, columns=['id_juego', 'genero_tag'])

df_juego_genero_tag['genero_tag'].value_counts().reset_index().to_csv('genero_tag.csv')

In [160]:
generos_filtrados = df_juego_genero_tag['genero_tag'].value_counts().head(50).reset_index().head(38)['genero_tag'].to_list()

In [163]:
mask = df_juego_genero_tag['genero_tag'].isin(generos_filtrados)

In [165]:
df_juego_genero_tag['genero_tag'][mask]

0                Indie
1             Strategy
2           Simulation
3               Casual
4               Action
              ...     
155977           Indie
155978     Atmospheric
155980          Casual
155981          Puzzle
155982    Singleplayer
Name: genero_tag, Length: 110776, dtype: object

In [167]:
a_clustear = pd.get_dummies(df_juego_genero_tag['genero_tag'][mask], dtype='int')

In [187]:
a_clustear.shape

(110776, 38)

In [181]:
df_games.query('id == 12500')

,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer,hash,genres_tag
109,"D3Publisher of America, Inc.",[Casual],PuzzleQuest: Challenge of the Warlords,2007-10-10,"[Puzzle, Casual, Match 3, RPG, Fantasy, 2D, St...","[Single-player, Multi-player, Steam Cloud]",9.99,False,12500,Infinite Interactive,3e77e82a28769f72bb048bf9ce4eff8c,"[Casual, Puzzle, Casual, Match 3, RPG, Fantasy..."


In [173]:
from sklearn.cluster import KMeans
import numpy as np

# Assuming you have your data stored in a variable called 'data'

# Create a KMeans instance with 38 clusters
kmeans = KMeans(n_clusters=38)

# Fit the KMeans model to your data
kmeans.fit(a_clustear)

# Get the cluster labels for each data point
cluster_labels = kmeans.labels_

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

In [186]:
preds = kmeans.predict(a_clustear)

preds.shape

(110776,)

In [172]:
%pip install scikit-learn


  Using cached scikit_learn-1.4.1.post1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.12.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.4.1.post1-cp312-cp312-win_amd64.whl (10.6 MB)
Using cached scipy-1.12.0-cp312-cp312-win_amd64.whl (45.8 MB)
Using cached threadpoolctl-3.3.0-py3-none-any.whl (17 kB)
